In [ ]:
#importing openai libraries and install openai
import openai


!pip install jupyter_bokeh
!pip install openai==0.28
import panel as pn
pn.extension()
openai.api_key = input("Enter your OpenAI API key: ") # Use your own API key
# Initial system prompt contains lists of service agents which can also be passed as a csv file/database.
context = [{"role": "system", "content": '''You are RepairBot, an automated service to collect appointment requests for new customers.
    You first greet the customer, then collects the issues related to the mobile phone,
    and then asks if the issue is related with device hardware or software.
    You need to check if the issue is with regards to battery or display not turning on, device rebooting itself, not turning on etc, and decide if it's h/w or s/w.
    You wait to collect the entire information with regards to the issues, then summarize it and check for a final
    time if the customer wants to add anything else.
    Make sure to clarify all options, if they require door services or instore.
    Provide them appointment timings between 8am to 11pm in weekdays and 9am to 8pm on weekends.
    Book appointments based on availablility of service agents.
    Use list of service agents below and book appointments as per availability of customer.
    - Riya Sharma (ID: MSA-1021)
    - Monday: 8:00 AM – 2:00 PM
    - Tuesday: 9:00 AM – 3:00 PM
    - Wednesday: 10:00 AM – 4:00 PM
    - Saturday: 9:00 AM – 1:00 PM
    - Sunday: Off
    - Thursday & Friday: Off
    - Arjun Mehta (ID: MSA-1043)
    - Monday: 3:00 PM – 11:00 PM
    - Wednesday: 2:00 PM – 10:00 PM
    - Friday: 12:00 PM – 8:00 PM
    - Saturday: 10:00 AM – 6:00 PM
    - Sunday: 11:00 AM – 5:00 PM
    - Tuesday & Thursday: Off
    - Sneha Kapoor (ID: MSA-1078)
    - Tuesday: 10:00 AM – 6:00 PM
    - Wednesday: 8:00 AM – 2:00 PM
    - Thursday: 11:00 AM – 7:00 PM
    - Friday: 9:00 AM – 3:00 PM
    - Saturday: 12:00 PM – 8:00 PM
    - Monday & Sunday: Off
    - Devansh Rao (ID: MSA-1095)
    - Monday: 8:00 AM – 4:00 PM
    - Tuesday: 2:00 PM – 10:00 PM
    - Thursday: 10:00 AM – 6:00 PM
    - Saturday: 9:00 AM – 2:00 PM
    - Sunday: 2:00 PM – 7:00 PM
    - Wednesday & Friday: Off
    - Tanya Verma (ID: MSA-1102)
    - Monday: 11:00 AM – 7:00 PM
    - Wednesday: 3:00 PM – 11:00 PM
    - Friday: 10:00 AM – 6:00 PM
    - Saturday: 11:00 AM – 5:00 PM
    - Sunday: Off
    - Tuesday & Thursday: Off
    - Karan Singh (ID: MSA-1117)
    - Monday: 8:00 AM – 2:00 PM
    - Tuesday: 9:00 AM – 1:00 PM
    - Wednesday: 10:00 AM – 4:00 PM
    - Friday: 2:00 PM – 6:00 PM
    - Sunday: 9:00 AM – 12:00 PM
    - Thursday & Saturday: Off
    - Mehul Joshi (ID: MSA-1130)
    - Tuesday: 1:00 PM – 9:00 PM
    - Wednesday: 2:00 PM – 10:00 PM
    - Thursday: 3:00 PM – 11:00 PM
    - Saturday: 1:00 PM – 7:00 PM
    - Sunday: 10:00 AM – 4:00 PM
    - Monday & Friday: Off
    - Priya Nair (ID: MSA-1144)
    - Monday: 10:00 AM – 6:00 PM
    - Wednesday: 9:00 AM – 3:00 PM
    - Thursday: 12:00 PM – 8:00 PM
    - Saturday: 3:00 PM – 8:00 PM
    - Sunday: Off
    - Tuesday & Friday: Off
    - Aditya Chauhan (ID: MSA-1159)
    - Monday: 1:00 PM – 9:00 PM
    - Tuesday: 2:00 PM – 10:00 PM
    - Wednesday: 3:00 PM – 11:00 PM
    - Saturday: 10:00 AM – 3:00 PM
    - Sunday: 12:00 PM – 6:00 PM
    - Thursday & Friday: Off
    - Neha Bansal (ID: MSA-1173)
    - Tuesday: 12:00 PM – 8:00 PM
    - Wednesday: 1:00 PM – 9:00 PM
    - Thursday: 2:00 PM – 10:00 PM
    - Friday: 3:00 PM – 11:00 PM
    - Saturday: 11:00 AM – 6:00 PM
    - Monday & Sunday: Off
    Door service requires extra 10 - 50 based on location.
    identify the issues with the phone from the list.
    If user is not sure or does not know what exactly the issue needs to be diagnosed by technician ask user to send contact details and provide local address.\
    You respond in a short, very conversational friendly style.
    Also add diagnostic fees for 50.
    The list includes
    display issues 100 for budget phones, 1000 for flagships
    battery issues 20 for budget phones, 55 for flagships
    software issues 30
    Once repair job is done request for customer feedback for agent service,
    request for rating between 1 to 5 and add survey ratings between 1 to 5 to find quality of service,
    request additional feedback through chat'''}]

# Widgets
inp = pn.widgets.TextInput(placeholder="Type your message…")
button = pn.widgets.Button(name="Send", button_type="primary")
chat_display = pn.Column(scroll=True, height=400)

# OpenAI call
def get_response(messages):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages
        )
        return response.choices[0].message["content"]
    except Exception as e:
        return f"Error: {str(e)}"

# Callback
def update_chat(event):
    user_msg = inp.value.strip()
    if not user_msg:
        return
    inp.value = ""
    context.append({"role": "user", "content": user_msg})
    bot_msg = get_response(context)
    context.append({"role": "assistant", "content": bot_msg})
    chat_display.append(pn.pane.Markdown(f"**You:** {user_msg}"))
    chat_display.append(pn.pane.Markdown(f"**Bot:** {bot_msg}"))

button.on_click(update_chat)

# Layout
app = pn.Column("# 🤖 RepairBot", chat_display, pn.Row(inp, button))
app.servable()

In [ ]:
import json
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message["content"]

# Copy context to avoid modifying original
messages = context.copy()

# Add system instruction to generate structured JSON
messages.append({
    'role': 'system',
    'content': '''Create a JSON summary of the previous issue. Itemize the price for each issue.
The fields should be:
1) phone type
2) type of service (door or instore)
3) appointment time
4) type of issue (hardware or software)
5) agent name, ID, and time of appointment
6) total estimated price including diagnostic fees
7) agent rating
8) quality of service rating
9) feedback'''
})

# Get response
response = get_completion_from_messages(messages, temperature=0)

# Try parsing to confirm it's valid JSON
try:
    parsed = json.loads(response)
    print(json.dumps(parsed, indent=2))
except json.JSONDecodeError:
    print("⚠️ Response was not valid JSON:\n", response)

⚠️ Response was not valid JSON:
 ```json
{
    "phone type": "Not specified",
    "type of service": "In-store",
    "appointment time": "Tuesday, 3:00 PM - 11:00 PM",
    "type of issue": "Not sure (Hardware)",
    "agent": {
        "name": "Arjun Mehta",
        "ID": "MSA-1043",
        "appointment time": "Tuesday, 3:00 PM - 11:00 PM"
    },
    "total estimated price": "$60 (Diagnostic fees: $50, Battery issue: $10)",
    "agent rating": "To be provided after service",
    "quality of service rating": "To be provided after service",
    "feedback": "To be provided after service"
}
```


In [ ]:
import json
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message["content"]

# Copy context to avoid modifying original
messages = context.copy()

# Add system instruction to generate structured JSON
messages.append({
    'role': 'system',
    'content': '''Create a JSON summary of the previous issue. Itemize the price for each issue.
The fields should be:
1) phone type
2) type of service (door or instore)
3) appointment time
4) type of issue (hardware or software)
5) agent name, ID, and time of appointment
6) total estimated price including diagnostic fees
7) agent rating
8) quality of service rating
9) feedback'''
})

# Get response
response = get_completion_from_messages(messages, temperature=0)

# Try parsing to confirm it's valid JSON
try:
    parsed = json.loads(response)
    print(json.dumps(parsed, indent=2))
except json.JSONDecodeError:
    print("⚠️ Response was not valid JSON:\n", response)

⚠️ Response was not valid JSON:
 ```json
{
    "phone_type": "Not specified",
    "service_type": "In-store",
    "appointment_time": "Tuesday, 3:00 PM - 11:00 PM",
    "issue_type": "Hardware (Phone not charging)",
    "agent": {
        "name": "Arjun Mehta",
        "ID": "MSA-1043",
        "appointment_time": "Tuesday, 3:00 PM - 11:00 PM"
    },
    "total_estimated_price": "$60 (Diagnostic fees: $50, Hardware issue: $10)",
    "agent_rating": 4,
    "quality_of_service_rating": "Not provided",
    "feedback": "Arjun was amazing and fixed my issue in minutes"
}
```
